In [2]:
import requests
import fake_useragent
from typing import List
import pandas as pd
import  numpy as np

import json
import time

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import re

## Загрузка данных

In [25]:
profs = pd.read_csv("../repositories/proffesions.csv").values

In [33]:
driver = webdriver.Firefox(executable_path='./repositories/geckodriver')
oprions = webdriver.Firefox()


def get_info_by_vacancies(
        job_title: str, 
        driver
    ) -> dict[str, dict[str, int]]:
    """
    Функция принимающая на вход профессию, и список всех регионов в России.
    Возвращает: {
        "job_title": {
            "mean_salary": int,
            "count_vacancies": int
    }}
    """
    def get_all_regions() -> List[str]:
        """
        Функция возвращающая, номера всех регионов в России.
        """
        user_agent = fake_useragent.UserAgent()
        regions = requests.get(
            url="https://api.hh.ru/areas/113",
            headers={"user_agent": user_agent.random}
        ).json()
        return [int(region["id"]) for region in regions["areas"]]
    
    def get_count_vacancies(job_title: str) -> int:
        url = fr'https://hh.ru/search/vacancy?text={job_title}'
        driver.get(url)

        el = driver.find_element(By.XPATH, "//h1[@data-qa='bloko-header-3']")
        return int(''.join(re.findall(r'\d+', el.text)))
    
    user_agent = fake_useragent.UserAgent()
    mean_salary_by_region = []

    result_row = {}

    regions = get_all_regions()
    for region in regions:
        url = f"https://api.hh.ru/vacancies?clusters=true&only_with_salary=true&enable_snippets=true&st=searchVacancy' \
            '&text={job_title}&search_field=name&per_page=100&area={region}"
        
        headers = requests.utils.default_headers()
        headers.update({"User-Agent": user_agent.random})
        vacancies = requests.get(
            url=url,
            headers=headers                   
        ).json()

        for vacancy in vacancies["items"]:
            if vacancy["salary"]["from"] is None:
                continue
    mean_salary_by_region.append(vacancy["salary"]["from"])
    result_row[job_title] = {
        "Средняя зарплата.": mean_salary_by_region,
        "Количество вакансий в России.": get_count_vacancies(job_title=job_title)
    }
    return result_row

/var/folders/ln/l54hjc713kxgxkw6rfxxgyh00000gn/T/ipykernel_1451/3722622066.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path='./repositories/geckodriver')


In [34]:
for prof in profs[:10]:
    print(get_info_by_vacancies(job_title=prof, driver=driver))

UnboundLocalError: local variable 'vacancy' referenced before assignment

,Психолог-консультант,R-разработчик,Руководитель отдела кадров,Аналитик BI,Оператор дрона,Product Marketing Manager,Data Engineer,event-менеджер,Специалист по Machine Learning,Музыкальный продюсер,...,Учитель-дефектолог,Bitrix-разработчик,Флорист,Фрилансер,Операционный менеджер,Педагог предшкольной подготовки,Фотограф интерьера,SRE-инженер,Категорийный менеджер,Пилот дрона
0,"{'Средняя зарплата.': 35000, 'Количество вакан...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,"{'Средняя зарплата.': 143333, 'Количество вака...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,"{'Средняя зарплата.': 61666, 'Количество вакан...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,"{'Средняя зарплата.': 0, 'Количество вакансий ...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,"{'Средняя зарплата.': 0, 'Количество вакансий ...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"{'Средняя зарплата.': 0, 'Количество вакансий ...",NaN,NaN,NaN,NaN
462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'Средняя зарплата.': 0, 'Количество вакансий ...",NaN,NaN,NaN
463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Средняя зарплата.': 0, 'Количество вакансий ...",NaN,NaN
464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Средняя зарплата.': 100000, 'Количество вака...",NaN


## Формирование excel файла

In [3]:
with open('../repositories/output.json') as data_file:
    data = json.load(data_file)

In [4]:
pd.DataFrame(data)


,VacanciesName,CountVacancies,MeanSalary
0,Психолог-консультант,53333,587
1,R-разработчик,0,Не найденно.


,Название профессии,Количество вакансий по профессиии,Средняя зарплата
0,1,1,1
1,1,1,1
2,1,1,1
3,1,1,1
4,1,1,1
5,1,1,1
